<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [83]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [84]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [85]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [86]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)


(array([0, 1]), array([619, 619]))

### Track Experiments

In [87]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [88]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [89]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [90]:
# Initialize MLflow (URI FIRST, then experiment)

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Anomaly Detection4")

for i, element in enumerate(models):
    # adjust these lines to match your models structure
    model_name, params, model = element[:3]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        # params must be a dict; if you meant model hyperparams:
        # mlflow.log_params(model.get_params())
        mlflow.log_params(params)

        mlflow.log_metrics({
            "accuracy": report["accuracy"],
            "recall_class_1": report["1"]["recall"],
            "recall_class_0": report["0"]["recall"],
            "f1_score_macro": report["macro avg"]["f1-score"],
        })

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")


2025/08/17 16:23:09 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection4' does not exist. Creating a new experiment.
2025/08/17 16:23:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/17 16:23:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/17 16:23:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/783244763926665061/runs/4ba1125a6f954fd29eb39a9109f31b92
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/783244763926665061


2025/08/17 16:23:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/17 16:23:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/783244763926665061/runs/51f4fc7f63ff490e98c5fcbc4f78ee51
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/783244763926665061


2025/08/17 16:23:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/17 16:23:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/783244763926665061/runs/028b877292d04182b018e6a4650c57ad
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/783244763926665061


2025/08/17 16:23:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/783244763926665061/runs/d3b22291ef3c4a1ba32b257c828403a4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/783244763926665061


### Register the Model

In [91]:
run_id = input("Please type RunID: ").strip()
model_uri = f"runs:/{run_id}/model"
result = mlflow.register_model(model_uri=model_uri, name="XGB-Smote")
print("Registered version:", result.version)


Please type RunID:  d3b22291ef3c4a1ba32b257c828403a4


Successfully registered model 'XGB-Smote'.
2025/08/17 16:23:26 WARNING mlflow.tracking._model_registry.fluent: Run with id d3b22291ef3c4a1ba32b257c828403a4 has no artifacts at artifact path 'model', registering model based on models:/m-47d46c0393d74f238d84b8d24e98dad6 instead
2025/08/17 16:23:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1


Registered version: 1


Created version '1' of model 'XGB-Smote'.


Model name: XGB-Smote
Version: 1
Source URI: models:/m-47d46c0393d74f238d84b8d24e98dad6
Run ID: d3b22291ef3c4a1ba32b257c828403a4


Loaded via pyfunc OK


### Load the Model

In [95]:
import mlflow
from mlflow.tracking import MlflowClient

# 1) Make sure you're talking to the same tracking/registry as where you registered
# mlflow.set_tracking_uri("YOUR_TRACKING_OR_REGISTRY_URI")  # uncomment if you use a custom URI

model_name = "XGB-Smote"          # must match exactly
model_version = "1"               # as a string or int is fine
model_uri = f"models:/{model_name}/{model_version}"

# 2) Verify the registry entry exists and see where it points
client = MlflowClient()
mv = client.get_model_version(model_name, model_version)
print("Model name:", mv.name)
print("Version:", mv.version)
print("Source URI:", mv.source)   # where artifacts live
print("Run ID:", mv.run_id)

# 3) Load robustly (pyfunc works regardless of flavor), then predict
loaded = mlflow.pyfunc.load_model(model_uri)
print("Loaded via pyfunc OK")


Model name: XGB-Smote
Version: 1
Source URI: models:/m-47d46c0393d74f238d84b8d24e98dad6
Run ID: d3b22291ef3c4a1ba32b257c828403a4


Loaded via pyfunc OK


### Transition the Model to Production

In [96]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

MlflowException: Failed to fetch model version from source model URI: 'models:/XGB-Smote@challenger'. Error: INVALID_PARAMETER_VALUE: Registered model alias challenger not found.

In [97]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

RestException: RESOURCE_DOES_NOT_EXIST: Registered Model with name=anomaly-detection-prod not found

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 